In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

### Reading the data

In [8]:
%%time
df = spark.read.csv("./data/gpu.csv", header=True, inferSchema=True)
#df


Wall time: 2.3 s


In [3]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- hostname: string (nullable = true)
 |-- gpuSerial: long (nullable = true)
 |-- gpuUUID: string (nullable = true)
 |-- powerDrawWatt: double (nullable = true)
 |-- gpuTempC: integer (nullable = true)
 |-- gpuUtilPerc: integer (nullable = true)
 |-- gpuMemUtilPerc: integer (nullable = true)



#### checking dataframe

In [4]:
df.show()
%time df.show()

+--------------------+--------------------+------------+--------------------+-------------+--------+-----------+--------------+
|           timestamp|            hostname|   gpuSerial|             gpuUUID|powerDrawWatt|gpuTempC|gpuUtilPerc|gpuMemUtilPerc|
+--------------------+--------------------+------------+--------------------+-------------+--------+-----------+--------------+
|2018-11-08 08:27:...|8b6a0eebc87b4cb2b...|323217055910|GPU-1d1602dc-f615...|       131.55|      48|         92|            53|
|2018-11-08 08:27:...|d8241877cd994572b...|323617020295|GPU-04a2dea7-f4f1...|       117.03|      40|         92|            48|
|2018-11-08 08:27:...|db871cd77a544e13b...|323217056562|GPU-f4597939-a0b4...|       121.64|      45|         91|            44|
|2018-11-08 08:27:...|b9a1fa7ae2f74eb68...|325217085931|GPU-ad773c69-c386...|        50.23|      38|         90|            43|
|2018-11-08 08:27:...|db871cd77a544e13b...|323217056464|GPU-2d4eed64-4ca8...|       141.82|      41|    

In [5]:
df.head(200)
%time df.head(200)

Wall time: 291 ms


[Row(timestamp=datetime.datetime(2018, 11, 8, 8, 27, 10, 314000), hostname='8b6a0eebc87b4cb2b0539e81075191b900001C', gpuSerial=323217055910, gpuUUID='GPU-1d1602dc-f615-a7c7-ab53-fb4a7a479534', powerDrawWatt=131.55, gpuTempC=48, gpuUtilPerc=92, gpuMemUtilPerc=53),
 Row(timestamp=datetime.datetime(2018, 11, 8, 8, 27, 10, 192000), hostname='d8241877cd994572b46c861e5d144c85000000', gpuSerial=323617020295, gpuUUID='GPU-04a2dea7-f4f1-12d0-b94d-996446746e6f', powerDrawWatt=117.03, gpuTempC=40, gpuUtilPerc=92, gpuMemUtilPerc=48),
 Row(timestamp=datetime.datetime(2018, 11, 8, 8, 27, 10, 842000), hostname='db871cd77a544e13bc791a64a0c8ed50000006', gpuSerial=323217056562, gpuUUID='GPU-f4597939-a0b4-e78a-2436-12dbab9a350f', powerDrawWatt=121.64, gpuTempC=45, gpuUtilPerc=91, gpuMemUtilPerc=44),
 Row(timestamp=datetime.datetime(2018, 11, 8, 8, 27, 10, 424000), hostname='b9a1fa7ae2f74eb68f25f607980f97d7000010', gpuSerial=325217085931, gpuUUID='GPU-ad773c69-c386-a4be-b214-1ea4fc6045df', powerDrawWatt=5

In [6]:
df.show(200)
%time df.show(200)

+--------------------+--------------------+------------+--------------------+-------------+--------+-----------+--------------+
|           timestamp|            hostname|   gpuSerial|             gpuUUID|powerDrawWatt|gpuTempC|gpuUtilPerc|gpuMemUtilPerc|
+--------------------+--------------------+------------+--------------------+-------------+--------+-----------+--------------+
|2018-11-08 08:27:...|8b6a0eebc87b4cb2b...|323217055910|GPU-1d1602dc-f615...|       131.55|      48|         92|            53|
|2018-11-08 08:27:...|d8241877cd994572b...|323617020295|GPU-04a2dea7-f4f1...|       117.03|      40|         92|            48|
|2018-11-08 08:27:...|db871cd77a544e13b...|323217056562|GPU-f4597939-a0b4...|       121.64|      45|         91|            44|
|2018-11-08 08:27:...|b9a1fa7ae2f74eb68...|325217085931|GPU-ad773c69-c386...|        50.23|      38|         90|            43|
|2018-11-08 08:27:...|db871cd77a544e13b...|323217056464|GPU-2d4eed64-4ca8...|       141.82|      41|    

+--------------------+--------------------+------------+--------------------+-------------+--------+-----------+--------------+
|           timestamp|            hostname|   gpuSerial|             gpuUUID|powerDrawWatt|gpuTempC|gpuUtilPerc|gpuMemUtilPerc|
+--------------------+--------------------+------------+--------------------+-------------+--------+-----------+--------------+
|2018-11-08 08:27:...|8b6a0eebc87b4cb2b...|323217055910|GPU-1d1602dc-f615...|       131.55|      48|         92|            53|
|2018-11-08 08:27:...|d8241877cd994572b...|323617020295|GPU-04a2dea7-f4f1...|       117.03|      40|         92|            48|
|2018-11-08 08:27:...|db871cd77a544e13b...|323217056562|GPU-f4597939-a0b4...|       121.64|      45|         91|            44|
|2018-11-08 08:27:...|b9a1fa7ae2f74eb68...|325217085931|GPU-ad773c69-c386...|        50.23|      38|         90|            43|
|2018-11-08 08:27:...|db871cd77a544e13b...|323217056464|GPU-2d4eed64-4ca8...|       141.82|      41|    

In [4]:
import pyspark.sql.functions

In [6]:
%%time
import pyspark.sql.functions as F
split_col = pyspark.sql.functions.split(df['timestamp'], ' ')
df_date = df.withColumn('Date', split_col.getItem(0))
df_time = df.withColumn('Time', F.concat(split_col.getItem(1),F.lit(' '),split_col.getItem(2)))

Wall time: 53.8 ms


In [7]:
# Select everybody, but increment the age by 1
df.select(df['gpuTempC'], df['gpuUtilPerc'], df['gpuMemUtilPerc'] + 1).show()
%time df.select(df['gpuTempC'], df['gpuUtilPerc'], df['gpuMemUtilPerc'] + 1) 

+--------+-----------+--------------------+
|gpuTempC|gpuUtilPerc|(gpuMemUtilPerc + 1)|
+--------+-----------+--------------------+
|      48|         92|                  54|
|      40|         92|                  49|
|      45|         91|                  45|
|      38|         90|                  44|
|      41|         90|                  48|
|      43|         88|                  41|
|      41|         91|                  48|
|      35|          0|                   1|
|      43|         93|                  57|
|      36|         90|                  41|
|      40|         91|                  49|
|      43|         90|                  45|
|      41|          0|                   1|
|      39|         90|                  49|
|      42|         92|                  47|
|      43|         93|                  53|
|      42|         92|                  42|
|      51|         88|                  44|
|      41|         89|                  43|
|      44|         91|          

DataFrame[gpuTempC: int, gpuUtilPerc: int, (gpuMemUtilPerc + 1): int]

In [8]:
df.select("gpuTempC").show()
%time df.select("gpuTempC")

+--------+
|gpuTempC|
+--------+
|      48|
|      40|
|      45|
|      38|
|      41|
|      43|
|      41|
|      35|
|      43|
|      36|
|      40|
|      43|
|      41|
|      39|
|      42|
|      43|
|      42|
|      51|
|      41|
|      44|
+--------+
only showing top 20 rows

Wall time: 9.97 ms


DataFrame[gpuTempC: int]

In [4]:
%%time
df.groupBy("gpuTempC").count()


Wall time: 16.8 ms


DataFrame[gpuTempC: int, count: bigint]

In [6]:
%%time
df.groupBy("gpuTempC").mean( "gpuTempC") 


Wall time: 24.9 ms


DataFrame[gpuTempC: int, avg(gpuTempC): double]

#### Calculating stats

In [8]:
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
#, std as std_
df_stats = df.select(
    _mean(col('gpuTempC')).alias('mean'),
    ).collect()

mean = df_stats[0]['mean']



#df.select("gpuTempC").mean_().show()

In [9]:
%%time
df_stats = df.select(
    _mean(col('gpuTempC')).alias('mean'),
    ).collect()

mean = df_stats[0]['mean']
df_stats[0]['mean']

Wall time: 929 ms


40.07560435089892

In [11]:
 
df_stats[0]['mean']

40.07560435089892

In [12]:
%%time
df_stats = df.select(
    _stddev(col('gpuTempC')).alias('std')
).collect()

std = df_stats[0]['std']
df_stats[0]['std']

Wall time: 885 ms


3.8002432669827995

In [13]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("gpuTempC")

sqlDF = spark.sql("SELECT * FROM gpuTempC")
sqlDF.show()



+--------------------+--------------------+------------+--------------------+-------------+--------+-----------+--------------+
|           timestamp|            hostname|   gpuSerial|             gpuUUID|powerDrawWatt|gpuTempC|gpuUtilPerc|gpuMemUtilPerc|
+--------------------+--------------------+------------+--------------------+-------------+--------+-----------+--------------+
|2018-11-08 08:27:...|8b6a0eebc87b4cb2b...|323217055910|GPU-1d1602dc-f615...|       131.55|      48|         92|            53|
|2018-11-08 08:27:...|d8241877cd994572b...|323617020295|GPU-04a2dea7-f4f1...|       117.03|      40|         92|            48|
|2018-11-08 08:27:...|db871cd77a544e13b...|323217056562|GPU-f4597939-a0b4...|       121.64|      45|         91|            44|
|2018-11-08 08:27:...|b9a1fa7ae2f74eb68...|325217085931|GPU-ad773c69-c386...|        50.23|      38|         90|            43|
|2018-11-08 08:27:...|db871cd77a544e13b...|323217056464|GPU-2d4eed64-4ca8...|       141.82|      41|    

### Merge

In [7]:
%%time
df1 = spark.read.csv("./data/application-checkpoints.csv", header=True, inferSchema=True)
df2 = spark.read.csv("./data/task-x-y.csv", header=True, inferSchema=True)
df = spark.read.csv("./data/gpu.csv", header=True, inferSchema=True)

Wall time: 3.97 s


In [16]:
%%time
df_merged = df1.join(df2, on=['taskId'], how='left_outer')
#%time df_merged

Wall time: 17 ms


In [14]:
%%time
df_merged_extra = df1.join(df, on=['hostname'], how='left_outer')

Wall time: 48.9 ms


In [17]:
%%time
full_merge_3_files = df_merged.join(df, on=['hostname'], how='left_outer')

Wall time: 16 ms


In [16]:
df_merged.show()
%time df_merged.show()

+--------------------+--------------------+--------------------+-------------+---------+--------------------+--------------------+---+---+-----+
|              taskId|           timestamp|            hostname|    eventName|eventType|               jobId|               jobId|  x|  y|level|
+--------------------+--------------------+--------------------+-------------+---------+--------------------+--------------------+---+---+-----+
|b47f0263-ba1c-48a...|2018-11-08 07:41:...|0d56a730076643d58...|       Tiling|     STOP|1024-lvl12-7e026b...|1024-lvl12-7e026b...|  2| 10|   12|
|20fb9fcf-a927-4a4...|2018-11-08 07:42:...|0d56a730076643d58...|Saving Config|    START|1024-lvl12-7e026b...|1024-lvl12-7e026b...|238|  4|   12|
|20fb9fcf-a927-4a4...|2018-11-08 07:42:...|0d56a730076643d58...|Saving Config|     STOP|1024-lvl12-7e026b...|1024-lvl12-7e026b...|238|  4|   12|
|20fb9fcf-a927-4a4...|2018-11-08 07:42:...|0d56a730076643d58...|       Render|    START|1024-lvl12-7e026b...|1024-lvl12-7e026b...|

In [17]:
df_merged.show(400)
%time df_merged.show(400)

+--------------------+--------------------+--------------------+-------------+---------+--------------------+--------------------+---+---+-----+
|              taskId|           timestamp|            hostname|    eventName|eventType|               jobId|               jobId|  x|  y|level|
+--------------------+--------------------+--------------------+-------------+---------+--------------------+--------------------+---+---+-----+
|b47f0263-ba1c-48a...|2018-11-08 07:41:...|0d56a730076643d58...|       Tiling|     STOP|1024-lvl12-7e026b...|1024-lvl12-7e026b...|  2| 10|   12|
|20fb9fcf-a927-4a4...|2018-11-08 07:42:...|0d56a730076643d58...|Saving Config|    START|1024-lvl12-7e026b...|1024-lvl12-7e026b...|238|  4|   12|
|20fb9fcf-a927-4a4...|2018-11-08 07:42:...|0d56a730076643d58...|Saving Config|     STOP|1024-lvl12-7e026b...|1024-lvl12-7e026b...|238|  4|   12|
|20fb9fcf-a927-4a4...|2018-11-08 07:42:...|0d56a730076643d58...|       Render|    START|1024-lvl12-7e026b...|1024-lvl12-7e026b...|

+--------------------+--------------------+--------------------+-------------+---------+--------------------+--------------------+---+---+-----+
|              taskId|           timestamp|            hostname|    eventName|eventType|               jobId|               jobId|  x|  y|level|
+--------------------+--------------------+--------------------+-------------+---------+--------------------+--------------------+---+---+-----+
|b47f0263-ba1c-48a...|2018-11-08 07:41:...|0d56a730076643d58...|       Tiling|     STOP|1024-lvl12-7e026b...|1024-lvl12-7e026b...|  2| 10|   12|
|20fb9fcf-a927-4a4...|2018-11-08 07:42:...|0d56a730076643d58...|Saving Config|    START|1024-lvl12-7e026b...|1024-lvl12-7e026b...|238|  4|   12|
|20fb9fcf-a927-4a4...|2018-11-08 07:42:...|0d56a730076643d58...|Saving Config|     STOP|1024-lvl12-7e026b...|1024-lvl12-7e026b...|238|  4|   12|
|20fb9fcf-a927-4a4...|2018-11-08 07:42:...|0d56a730076643d58...|       Render|    START|1024-lvl12-7e026b...|1024-lvl12-7e026b...|

In [46]:
%%time
df_stats = df_merged.select(
    _mean(col('x')).alias('mean'),
    ).collect()

mean = df_stats[0]['mean']
df_stats[0]['mean']

Wall time: 2.76 s


127.01954875832828

In [47]:
%%time
df_stats = df_merged.select(
    _stddev(col('x')).alias('std')
).collect()

std = df_stats[0]['std']
df_stats[0]['std']

Wall time: 1.05 s


74.1352849931841